In [1]:
from jupyter_plotly_dash import JupyterDash
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import sqlite3
import pandas as pd
import plotly.express as px

In [2]:
dbPath = "https://github.com/AlvinLeongWS/malaria/malaria.db"
def getData(sqlStr):
    conn = sqlite3.connect(dbPath) 
    df = pd.read_sql_query(sqlStr, conn)
    return df

In [33]:
def createBarChart(data, xAxis, yAxis, color, labels = None, title=None):
    fig = px.bar(data, x=xAxis, y=yAxis, color=color, text=yAxis, labels = labels, title = title)
    return fig
def createPieChart(data, name, value, color, labels = None, title=None):
    return px.pie(data, names = name, values = value, color=color, labels = labels, title = title)

def convertToList(df, columnName):
    list = []
    for index, row in df.iterrows():
        dict = {}
        dict["label"] = row[columnName]
        dict["value"] = row[columnName]
        list.append(dict) 
    return list

In [34]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# external_stylesheets = ['./plotly/css/style.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
app = JupyterDash('Example')

sqlStr = "Select Distinct Entity from Entity;"
Entitydf = getData(sqlStr)
sqlStr = "Select Entity, Year, Sum(DeathsRate) As DeathRate from malaria_death Group By Entity, Year;"
df1 = getData(sqlStr)
sqlStr = "Select a.Entity, a.Year, b.AgeGroup, a.NoOfDeaths from malaria_deaths_age a, " \
    "AgeGroup b Where a.AgeGroupID = b.ID"
agedf = getData(sqlStr)


EntityList = convertToList(Entitydf, "Entity")
EntityDropDown = dcc.Dropdown(id="EntityDD", options=EntityList, multi=True, searchable = True, 
                            placeholder="Select a entity")
tempdf = df1.groupby(["Year"]).sum().reset_index()
tempAgedf = agedf.groupby(["AgeGroup", "Year"]).sum().reset_index()

sqlStr = "Select Entity, Year, Sum(IncidenceOfMalaria) as IncidenceOfMalaria from malaria_inc Group By Entity, Year"
incByEntityYeardf = getData(sqlStr)
YearList = convertToList(pd.DataFrame(incByEntityYeardf["Year"].unique(), columns=["Year"]), "Year")
YearDropDown = dcc.Dropdown(id="YearDD", options=YearList, multi=True, searchable = True, 
                            placeholder="Select a year")

componentList = []
componentList.append(html.Div([EntityDropDown]))
componentList.append(html.Div([dcc.Graph(id="DeathByYear")]))
componentList.append(html.Div([dcc.Graph(id="DeathByAgeGroup")]))
marks={
        0: '0',
        5: '5',
        10: '10',
        15: '15',
        20: '20'
    }
componentList.append(html.Div([dcc.Slider(id='TopNumber',min=0,max=20, step=1, value=10,marks=marks)]))
componentList.append(html.Div([html.P()]))
componentList.append(html.Div([html.P()]))
componentList.append(html.Div([YearDropDown]))
tempTopdf = incByEntityYeardf.groupby(["Entity"]).sum().reset_index()
tempTopdf = tempTopdf.head(10)
componentList.append(html.Div([dcc.Graph(id="TopInc")]))


def appCallBack(app):
    @app.callback(
        dash.dependencies.Output('TopInc', 'figure'), 
        [dash.dependencies.Input('TopNumber', 'value'),
        dash.dependencies.Input('YearDD', 'value')])
    def updatePieChart(entities, YearValue):
        tempTopdf = None
        # return entities
        if YearValue is not None:
            if len(YearValue) > 0:
                tempTopdf = incByEntityYeardf[incByEntityYeardf["Year"].isin(YearValue)]
                tempTopdf = tempTopdf.groupby(["Entity"]).sum().reset_index()
                tempTopdf = tempTopdf.head(entities)
                print(tempTopdf)
            else:
                tempTopdf = incByEntityYeardf.groupby(["Entity"]).sum().reset_index()
                tempTopdf = tempTopdf.head(entities)
        else:
            tempTopdf = incByEntityYeardf.groupby(["Entity"]).sum().reset_index()
            tempTopdf = tempTopdf.head(entities)
        label = {
                     "Entity": "Entity"
                }
        return createPieChart(tempTopdf, "Entity", "IncidenceOfMalaria", "Entity", label , "Top Incidence By Entity")
        
    @app.callback(
        [dash.dependencies.Output('DeathByYear', 'figure'),
         dash.dependencies.Output('DeathByAgeGroup', 'figure')], 
        [dash.dependencies.Input('EntityDD', 'value')])
    def updateBarChart(value):
        tempdf = None
        if value is not None:
            if len(value) > 0:
                tempdf = df1[df1["Entity"].isin(value)]
                tempdf = tempdf.groupby(["Year"]).sum().reset_index()
                tempAgedf = agedf[agedf["Entity"].isin(value)]
                tempAgedf = tempAgedf.groupby(["AgeGroup", "Year"]).sum().reset_index()
            else:
                tempdf = df1.groupby(["Year"]).sum().reset_index()
                tempAgedf = agedf.groupby(["AgeGroup", "Year"]).sum().reset_index() 
        else:
            tempdf = df1.groupby(["Year"]).sum().reset_index()      
            tempAgedf = agedf.groupby(["AgeGroup", "Year"]).sum().reset_index() 
        
        label1 = {
                     "Year": "Year",
                     "DeathRate": "Both - Age: Age-standardized (Rate) (per 100,000 people)"
                 }
        label2 = {
                     "Year": "Year",
                     "NoOfDeaths": "No Of Death",
                     "AgeGroup": "Age Group"
                 }
        return [createBarChart(tempdf, "Year", "DeathRate", "Year", label1, "Deaths - Malaria - Sex"), 
                createBarChart(tempAgedf, "Year", "NoOfDeaths", "AgeGroup", label2, "Malaria death by age group")]
    
    return html.Div(componentList)
                                                                              
app.layout = appCallBack(app) 

In [35]:
app